In [2]:
import pandas as pd
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

cover_column_names=["Elevation","Aspect","Slope",
              "Horizontal_Distance_To_Hydrology","Vertical_Distance_To_Hydrology",
              "Horizontal_Distance_To_Roadways","Hillshade_9am","Hillshade_Noon",
              "Hillshade_3pm","Horizontal_Distance_To_Fire_Points"]
wilderness_cols =["Wilderness_"+str(i) for i in range(4)]
soil_col =["Soil_"+str(i) for i in range(40)]
cover_columns_names = cover_column_names + wilderness_cols + soil_col

COVER_FILE = './forest_cover_dataset.csv'
forest_dataFile = pd.read_csv(COVER_FILE,names=cover_columns_names)
#Determine the number and types of features.
forest_dataFile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 280251 entries, 0 to 280250
Data columns (total 54 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           280251 non-null  int64
 1   Aspect                              280251 non-null  int64
 2   Slope                               280251 non-null  int64
 3   Horizontal_Distance_To_Hydrology    280251 non-null  int64
 4   Vertical_Distance_To_Hydrology      280251 non-null  int64
 5   Horizontal_Distance_To_Roadways     280251 non-null  int64
 6   Hillshade_9am                       280251 non-null  int64
 7   Hillshade_Noon                      280251 non-null  int64
 8   Hillshade_3pm                       280251 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  280251 non-null  int64
 10  Wilderness_0                        280251 non-null  int64
 11  Wilderness_1                        280251 non-null 

In [3]:
forest_dataFile.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_30,Soil_31,Soil_32,Soil_33,Soil_34,Soil_35,Soil_36,Soil_37,Soil_38,Soil_39
0,2825,265,17,319,108,2298,174,245,209,342,...,0,0,1,0,0,0,0,0,0,0
1,2719,198,34,134,76,2352,187,245,167,693,...,0,0,0,0,0,0,0,0,0,0
2,3146,152,14,212,41,940,237,239,130,2007,...,0,0,0,0,0,0,0,0,0,0
3,2882,18,18,95,-3,485,203,199,128,342,...,1,0,0,0,0,0,0,0,0,0
4,2912,349,17,283,57,2729,187,210,160,2201,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Perform a ShuffleSplit of the data into training/validation/test sets, 60%/20%/20%.
from sklearn.model_selection import StratifiedShuffleSplit

def MySplit(n_splits,test_size,df):
  split = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=23721266)
  for train_index, test_valid_index in split.split(df, df['annual-income']):
     set1 = df.iloc[train_index]
     set2 = df.iloc[test_valid_index]
     return set1,set2

train_set,test_valid_set = MySplit(1,0.4,forest_dataFile)
test_set,valid_set = MySplit(1,0.5,test_valid_set)



In [ ]:
#Center the training/validation/test splits (fit on the training set, then transform the validation/test sets).


In [ ]:
#𝑘 -Means Clustering
from sklearn.cluster import KMeans

from joblib import Parallel, delayed, parallel_backend

with parallel_backend("loky", inner_max_num_threads=2):
    results = Parallel(n_jobs=4)(delayed(func)(x, y) for x, y in data)
    
k_values=list(range(2,11))+[15,20]

kmeans = KMeans(n_clusters=2, random_state=0).fit(train_set)